# Challenge 1 - T-test

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_re`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](https://researchbasics.education.uconn.edu/t-test/) and [this Quora](https://www.quora.com/What-is-the-difference-between-a-paired-and-unpaired-t-test). Make sure you understand when to use which type of t-test. 

In [1]:
# Import libraries

import pandas as pd

#### Import dataset

In this challenge we will work on the Pokemon dataset you have used last week. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.).

In [40]:
# Import dataset

pokemon = pd.read_csv('../data/Pokemon.csv')

pokemon.sample(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
778,708,Phantump,Ghost,Grass,309,43,70,48,50,60,38,6,False
71,65,AlakazamMega Alakazam,Psychic,NaN,590,55,50,65,175,95,150,1,False
510,460,Abomasnow,Grass,Ice,494,90,92,75,92,85,60,4,False
12,9,BlastoiseMega Blastoise,Water,NaN,630,79,103,120,135,115,78,1,False
563,504,Patrat,Normal,NaN,255,45,55,39,35,39,42,5,False
118,110,Weezing,Poison,NaN,490,65,90,120,85,70,60,1,False
107,99,Kingler,Water,NaN,475,55,130,115,50,50,75,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
623,562,Yamask,Ghost,NaN,303,38,30,85,55,65,30,5,False
271,251,Celebi,Psychic,Grass,600,100,100,100,100,100,100,2,False


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [3]:
from scipy import stats

def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    results = {}

    # Your code here

    for feature in features:
        data1 = s1[feature].dropna()
        data2 = s2[feature].dropna()

        t_stat, p_value = stats.ttest_ind(data1, data2, equal_var = False)

        results[feature] = p_value
    
    return results

In [19]:
# Your code here

sample1 = pokemon[pokemon['Legendary'] == True].sample(30)
sample2 = pokemon[pokemon['Legendary'] == False].sample(30)

t_test_features(sample1, sample2)

{'HP': 0.00029450942668019805,
 'Attack': 0.00019372974075130317,
 'Defense': 0.003515175355131006,
 'Sp. Atk': 6.036925647000889e-07,
 'Sp. Def': 0.00023770795747802992,
 'Speed': 3.3865981356130216e-08,
 'Total': 8.417507274268674e-14}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [ ]:
# Your comment here
# all p-values are low, so it means there are statistical differences between legendary and non-legendary pokemon

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [20]:
# Your code here

sample1 = pokemon[pokemon['Generation'] == 1].sample(30)
sample2 = pokemon[pokemon['Generation'] == 2].sample(30)

t_test_features(sample1, sample2)

{'HP': 0.8061970517753202,
 'Attack': 0.6741167507118822,
 'Defense': 0.8169845713672941,
 'Sp. Atk': 0.18439603916840122,
 'Sp. Def': 0.0070510418128881544,
 'Speed': 0.5390749438867943,
 'Total': 0.25042577477781414}

#### What conclusions can you make?

In [ ]:
# Your comment here

# all p-values are not below 0.05, so it means there is no statistical difference between first and second generation pokemon

#### Compare pokemons who have single type vs those having two types.

In [33]:
# Your code here

sample1 = pokemon[pokemon['Type 2'].notna()].sample(30)
sample2 = pokemon[pokemon['Type 2'].isna()].sample(30)

t_test_features(sample1, sample2)

{'HP': 0.487185583626787,
 'Attack': 0.0008863546785784968,
 'Defense': 0.0014020848319785835,
 'Sp. Atk': 0.24684175120820195,
 'Sp. Def': 0.006073785994169631,
 'Speed': 0.5045129433282234,
 'Total': 0.0014315949374409508}

#### What conclusions can you make?

In [ ]:
# Your comment here

# The differences between Pokémon with one type and Pokémon with two types seem to be particularly relevant for Attack and Defense characteristics, as p-values indicate significant differences in more than one set of outcomes.

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [39]:
# Your code here

print(stats.ttest_rel(pokemon['Attack'], pokemon['Defense']))
stats.ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])


TtestResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05, df=799)


TtestResult(statistic=0.853986188453353, pvalue=0.3933685997548122, df=799)

#### What conclusions can you make?

In [ ]:
# Your comment here

# the p-value of attack and defense is less than 0.05 so it means that there are differences among all pokemon between these two features. In contrast, Sp. Atk and Sp. Def have a higher p-value so it does not reject the null hypothesis